In [2]:
# imported dependencies
import pennylane as qml
from itertools import product
import numpy as np
import random


In [3]:
def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([0.5, 0, 0.5, 0, 0, 0.5, 0, 0.5])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

In [4]:
L = 3
rho_naught = generate_density_matrix()
estimation_wires = list(range(0, L))
target_wires = list(range(L, L+3))

dev = qml.device("default.mixed", wires=(len(estimation_wires+target_wires)))

@qml.qnode(dev)
def VQED(stabilizer_list, unitary_list):
    # stabilizer list is same len as estimation wires

    # initialize target and estimation wires
    qml.QubitDensityMatrix(rho_naught, wires=target_wires)   
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)

    for l in estimation_wires:
        # get the 3 unitary matries 
        s_i = stabilizer_list[l][0]
        s_j = stabilizer_list[l][1]
        U = unitary_list[l]
    
        # apply the gates 
        qml.QubitUnitary(U, wires=target_wires)
        qml.BitFlip(0.5,wires=[0])
        qml.ControlledQubitUnitary(s_i, control_wires=estimation_wires[l], wires=target_wires, control_values="0")
        qml.ControlledQubitUnitary(s_j, control_wires=estimation_wires[l], wires=target_wires, control_values="1")

    return qml.expval(qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliX(2)), qml.expval(qml.Hadamard(3) @ qml.PauliX(4) @ qml.Hadamard(5))


In [5]:
U1 = qml.PauliZ(0) @ qml.Identity(1) @ qml.Identity(2)
U1 = U1.matrix()
U2 = qml.Identity(0) @ qml.PauliZ(1) @ qml.Identity(2)
U2 = U2.matrix()
U3 = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
U3 = U3.matrix()
U_list = [U1, U2, U3]
# L = 3

G1 = (qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)).matrix()
G2 = (qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)).matrix()
I = (qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)).matrix()
S = [G1, G2, G1 @ G2, I]

a = 0
b = 0
N = 4
for s in range(N):
    s_sample = [random.sample(S,2),random.sample(S,2),random.sample(S,2)]
    a_s, O_measurement = VQED(s_sample, U_list)
    b_s = a_s * O_measurement
    a += a_s
    b += b_s

a = a/N
b = b/N
print(a)
print(b)
print(b/a)

0.25000000000001454
-0.12500000000001457
-0.5000000000000292
